In [3]:
import cv2
import numpy as np
from glob import glob

## Apply distortion to linear images generated in CoppeliaSim

### Build the intrinsic matrix of visual sensor

In [4]:
def build_intrinsic_matrix(fov_degrees, resolution):
    fov_radians = np.radians(fov_degrees)

    f_x = resolution[0]/(2*np.tan(fov_radians/2))
    f_y = resolution[1]/(2*np.tan(fov_radians/2))

    o_x = resolution[0]/2
    o_y = resolution[1]/2

    camera_matrix = np.array([[-f_x,   0, o_x, 0],
                              [  0, -f_y, o_y, 0],
                              [  0,    0,   1, 0]])

    return camera_matrix

# Declare intrinsic parameters
fov_degrees = 60.0
resolution = (480, 480)

# Generate intrinsic parameters matrix
intrinsic_matrix = build_intrinsic_matrix(fov_degrees, resolution) 

### Lens Distortion Models

In [5]:
def distort_rational(image_point, camera_matrix, distortion_coefficients):
    # Get intrinsic parameters
    f_x, f_y = camera_matrix[0][0], camera_matrix[1][1]
    c_x, c_y = camera_matrix[0][2], camera_matrix[1][2]

    [[u], [v]] = image_point

    # Normalize coordinates
    x, y = (u - c_x)/f_x, (v - c_y)/f_y 

    normalized_image_point = np.array([[x],
                                       [y]])

    # Radial distance
    r = np.linalg.norm(normalized_image_point)

    # Get distortion coefficients (OpenCV's style)
    k1, k2, p1, p2, k3, k4, k5, k6 = distortion_coefficients
    
    # Get radial and tangential transformation vectors
    radial_transformation = normalized_image_point * (1 + k1*r**2 + k2*r**4 + k3*r**6)/(1 + k4*r**2 + k5*r**4 + k6*r**6)

    tangential_transformationn = np.array([[2*p1*x*y + p2*(r**2 + 2*x**2)],
                                           [p1*(r**2 + 2*y**2) + 2*p2*x*y]])
    
    # Get distorted normalized point
    [[x_d], [y_d]] = radial_transformation + tangential_transformationn

    # Re-scale and re-center point
    u_d, v_d = x_d * f_x + c_x, y_d * f_y + c_y

    return np.array([[u_d],
                     [v_d]]).astype(int) # Cast as interger

In [6]:
def distort_fisheye(image_point, camera_matrix, fisheye_coefficients):
    # Get intrinsic parameters
    f_x, f_y = camera_matrix[0][0], camera_matrix[1][1]
    c_x, c_y = camera_matrix[0][2], camera_matrix[1][2]

    [[u], [v]] = image_point

    # Normalize coordinates
    x, y = (u - c_x)/f_x, (v - c_y)/f_y 

    normalized_image_point = np.array([[x],
                                       [y]])

    # Distortion parameters
    r = np.linalg.norm(normalized_image_point)
    theta = np.arctan(r)

    # Get distortion coefficients (OpenCV's style)
    k1, k2, k3, k4 = fisheye_coefficients

    if r != 0:
        [[x_d], [y_d]] = normalized_image_point * (theta + k1*theta**3 + k2*theta**5 + k3*theta**7 + k4*theta**9)/r
    else:
        [[x_d], [y_d]] = normalized_image_point # Do not distort

    # Re-scale and re-center point
    u_d, v_d = x_d * f_x + c_x, y_d * f_y + c_y

    return np.array([[u_d],
                     [v_d]]).astype(int) # Cast as interger

### Apply distortion in Coppelia's Images

In [7]:
# Arbitrary distortion coefficients for the two types of distortion
radial_coefficients = np.array([-0.11, -0.03, 0.00, 0, 0.0, 0.0])
tangential_coefficients = np.array([0, 0.00000])

# Separating the coefficients list into single variables
k1, k2, k3, k4, k5, k6 = radial_coefficients
p1, p2 = tangential_coefficients

# Ordering the coefficients for OpenCV's distortion coefficients array configuration
distortion_coefficients = np.array([k1, k2, p1, p2, k3, k4, k5, k6]).astype(np.float32)

# Arbitrary distortion coefficients for the distortion
fisheye_coefficients = np.array([0.395, 0.633, -2.417, 2.110]).astype(np.float32)

model = 'rational'

image_distorted = np.zeros((resolution[0], resolution[1], 3), dtype=np.uint8)
map_u = np.zeros((resolution[0], resolution[1], 1), dtype=np.float32)
map_v = np.zeros((resolution[0], resolution[1], 1), dtype=np.float32)

# Note that the u-axis represents the columns and the v-axis represents the rows
for v in range(resolution[0]):
    for u in range(resolution[1]):
        pixel_coordinate = np.array([[u],
                                     [v]])
        
        if model == 'rational':
            distorted_pixel_coordinate = distort_rational(image_point=pixel_coordinate, 
                                                         camera_matrix=intrinsic_matrix,
                                                         distortion_coefficients=distortion_coefficients)
        elif model == 'fisheye':
            distorted_pixel_coordinate = distort_fisheye(image_point=pixel_coordinate, 
                                                         camera_matrix=intrinsic_matrix,
                                                         fisheye_coefficients=fisheye_coefficients)

        [[u_d], [v_d]] = distorted_pixel_coordinate

        # Do not remap points outside the image limits
        if (u_d >= 0 and u_d < resolution[1]) and (v_d >= 0 and v_d < resolution[0]):
            map_u[v_d][u_d] = u
            map_v[v_d][u_d] = v

In [14]:
image_set = glob('../../images/virtual/7x7/original/test/*.jpg')

for idx,image in enumerate(image_set):
    image_linear = cv2.imread(image)
    # Use cv2.remap with the custom remapped coordinates
    image_distorted = cv2.remap(src=image_linear, 
                                map1=map_u, 
                                map2=map_v, 
                                interpolation=cv2.INTER_LINEAR,
                                borderMode=cv2.BORDER_WRAP)

    # Blur to remove distortion line patterns and ease blob detection 
    #kernel = np.ones((2,2), dtype=np.float32) / 2
    #image_distorted = cv2.filter2D(image_distorted, -1, kernel)

    # # Display the image
    # cv2.imshow("Image", image_distorted)

    # # Wait for the user to press a key
    # cv2.waitKey(0)
    
    # Close all windows
    # cv2.destroyAllWindows()
    
    cv2.imwrite(f'image {idx}.jpg', image_distorted)